# 残差网络

## 准备数据集

In [1]:
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim

batch_size = 64
# 使用torch把图像转化为tensor,图像 c * w * h
# 把单通道转换成多通道，把一系列的图像进行转换，变成 [0,1]之间。0.1307是均值，0.3081是标准差
# 使用Normalize也是希望像素值满足 0 1 分布
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,),(0.3081,))])
# 变换好之后直接放进数据集里
train_dataset = datasets.MNIST(root='./dataset/mnist',train=True,download=True,transform=transform)
train_loader = DataLoader(train_dataset,shuffle=True,batch_size=batch_size)
test_dataset = datasets.MNIST(root='./dataset/mnist',train=False,download=True,transform=transform)
test_loader = DataLoader(test_dataset,shuffle=False,batch_size=batch_size)

## 定义模型

### 定义残差模块

In [3]:
class ResidualBlock(torch.nn.Module):
    def __init__(self,channels):
        super(ResidualBlock,self).__init__()
        self.channels = channels
        # 为了保证输出图像大小不变，那么需要把padding设置为1.因为得保证输出和输入的通道得是一样的
        # padding的大小是通过kernel_size求的，3/2=1
        self.conv1 = torch.nn.Conv2d(in_channels=channels,out_channels=channels,kernel_size=3,padding=1)
        self.conv2 = torch.nn.Conv2d(in_channels=channels,out_channels=channels,kernel_size=3,padding=1)
        
    def forward(self,x):
        y = F.relu(self.conv1(x))
        y = self.conv2(y)
        return F.relu(x+y)
        

### 定义网络模型

In [4]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        # 先定义两个卷积层
        self.conv1 = torch.nn.Conv2d(1,16,kernel_size=5)
        self.conv2 = torch.nn.Conv2d(16,32,kernel_size=5)
        self.mp = torch.nn.MaxPool2d(2)
        
        self.rblock1 = ResidualBlock(16)
        self.rblock2 = ResidualBlock(32)
        
        self.fc = torch.nn.Linear(512,10)
        
    def forward(self,x):
        in_size = x.size(0)
        x = self.mp(F.relu(self.conv1(x)))
        x = self.rblock1(x)
        x = self.mp(F.relu(self.conv2(x)))
        x = self.rblock2(x)
        x = x.view(in_size,-1) 
        x = self.fc(x)
        return x
        
    
model = Net()

## 构造损失函数和优化器

In [5]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.01,momentum=0.5)

## 构造训练器

In [6]:
def train(epoch):
    running_loss = 0.0
    for batch_idx,data in enumerate(train_loader,0):
        inputs,target = data
        # 把inputs,target都迁移到GPU中
        # inputs,target = inputs.to(device),target.to(device)
        # 优化器在优化之前先清零
        optimizer.zero_grad()
        # forward backward update
        outputs = model(inputs)
        loss = criterion(outputs,target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        # 每300输出一次
        if batch_idx % 300 == 299:
            print('[%d,%5d] loss:%.3f' % (epoch+1,batch_idx+1,running_loss/300))
            # 每300一组数据跑完，清零
            running_loss = 0.0

## 测试

In [7]:
# test里面不需要计算反向传播，只需要计算正向的就行了
def test():
    # 表示正确了多少
    correct = 0
    # 表示总数有多少
    total = 0
    # 在test过程中是不需要计算梯度的
    # 这部分的代码不会求导
    with torch.no_grad():
        for data in test_loader:
            images,labels = data
            # 把数据迁移到GPU上
            # images,labels = images.to(device),labels.to(device)
            outputs = model(images)
            # dim=1 返回每一个行的最大值和索引
            # dim=0 返回每一列的最大值和索引
            # 这里的索引或者下标对应的是它的分类
            _,predicted = torch.max(outputs.data,dim=1)
            # label是一个元组，(N,1)
            # size是计算元素的个数，0表示计算行数，1表示计算列数
            total += labels.size(0)
            correct += (predicted==labels).sum().item()
            
    # 使用正确的数除以总数
    print('Accuracy on test set: %d %%' % (100 * correct/total))

In [8]:
for epoch in range(10):
    # 一轮训练
    train(epoch)
    # 一轮测试
    test()

[1,  300] loss:0.538
[1,  600] loss:0.152
[1,  900] loss:0.108
Accuracy on test set: 97 %
[2,  300] loss:0.092
[2,  600] loss:0.077
[2,  900] loss:0.073
Accuracy on test set: 98 %
[3,  300] loss:0.062
[3,  600] loss:0.061
[3,  900] loss:0.055
Accuracy on test set: 98 %
[4,  300] loss:0.053
[4,  600] loss:0.041
[4,  900] loss:0.050
Accuracy on test set: 98 %
[5,  300] loss:0.038
[5,  600] loss:0.042
[5,  900] loss:0.044
Accuracy on test set: 98 %
[6,  300] loss:0.037
[6,  600] loss:0.031
[6,  900] loss:0.039
Accuracy on test set: 98 %
[7,  300] loss:0.030
[7,  600] loss:0.031
[7,  900] loss:0.034
Accuracy on test set: 98 %
[8,  300] loss:0.027
[8,  600] loss:0.028
[8,  900] loss:0.029
Accuracy on test set: 98 %
[9,  300] loss:0.024
[9,  600] loss:0.025
[9,  900] loss:0.028
Accuracy on test set: 98 %
[10,  300] loss:0.022
[10,  600] loss:0.024
[10,  900] loss:0.022
Accuracy on test set: 98 %
